# Econ 240A, Part II, Fall 2017
## Problem Set 3, Question 3


In [4]:
# importing libraries

from __future__ import division
import numpy as np
import numpy.linalg as la
import pandas as pd
import statsmodels.api as sm


#importing data
data = pd.read_table('./Brazil_1996PNAD.out');

#taking out the data between ages 20 and 60
data = data[(data.AgeInDays <=60) & (data.AgeInDays >=20)];


# Drop observations with 0 earnings
data = data[data['MONTHLY_EARNINGS'] != 0];

# variables needed
data['LogEarn'] = np.log(data['MONTHLY_EARNINGS']);
data['AgeInDays2'] = data['AgeInDays']**2;


# Define OLS function
# Inputs: X = Pandas DataFrame
#         Y = Pandas DataFrame
# Output: output = Pandas DataFrame containing coef, se, and 95% CI
#         V  = covariance matrix
#         U  = residuals
def OLS(Y,X):
    
    # Obtain the list and number of covariates
    cols = X.columns;
    K = len(cols);

    # Convert to Numpy Arrays and construct inverse of (X^T X) and X^T Y
    X = np.array(X).reshape(-1,K);
    Y = np.array(Y);
    XT = X.T;
    G = la.inv(np.dot(XT,X));
    XY = np.dot(XT,Y);
    
    # Compute OLS coefficients
    coef = np.dot(G,XY);
    
    # Compute squre of residuals then create a diagonal matrix
    # Finally compute the sum of X_i U_i^2 X_i^T
    U = Y-np.dot(X,coef);
    XU = X*U;
    O = np.dot(XU.T,XU);
    
    # Compute covariance matrix
    V = np.dot(np.dot(G,O),G);

    # Compute Standard Error
    SE = np.sqrt(np.diag(V)).reshape(-1,1);
        
    # Compute 95% CI and display 4 decimal points
    lbnd = coef-1.96*SE;
    ubnd = coef+1.96*SE;
    CI = ['[%.5f,%.5f]' %(lbnd[i],ubnd[i]) for i in range(K)];
    CI =np.array(CI);
    
    # Put results together into a single Pandas DataFrame
    output = np.concatenate((coef,SE),axis=1);
    output = pd.DataFrame(output,index=cols);
    output.columns = ['Coef','SE'];
    output['95% CI'] = CI;
        
    V = pd.DataFrame(V,index=cols);
    V.columns = cols;
        
    return [output,V]







## Part (a)

In [5]:
# Part (a)Regression
    
# Define covariates and the dependent variable
Y = data[['LogEarn']];

X_list1 = ['YRSSCH','AgeInDays','AgeInDays2'];
X1 = data[X_list1];
X1 = sm.add_constant(X1);

# OLS fit
result_short = OLS(Y,X1);

# OLS fit using statsmodels.OLS
OLS_result = sm.OLS(Y,X1).fit(cov_type='HC0');

print '\n'
print 'Short Regression'
print result_short[0]
print '\n'
print 'Covariance Matrix'
print result_short[1]
print '\n'
print OLS_result.summary()





Short Regression
                Coef        SE               95% CI
const       3.044634  0.041665    [2.96297,3.12630]
YRSSCH      0.145612  0.000822    [0.14400,0.14722]
AgeInDays   0.089302  0.002340    [0.08472,0.09389]
AgeInDays2 -0.000903  0.000031  [-0.00096,-0.00084]


Covariance Matrix
                   const        YRSSCH     AgeInDays    AgeInDays2
const       1.736004e-03  8.284987e-07 -9.551517e-05  1.218483e-06
YRSSCH      8.284987e-07  6.758185e-07 -2.939232e-07  4.225726e-09
AgeInDays  -9.551517e-05 -2.939232e-07  5.477544e-06 -7.138706e-08
AgeInDays2  1.218483e-06  4.225726e-09 -7.138706e-08  9.481571e-10


                            OLS Regression Results                            
Dep. Variable:                LogEarn   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                 1.246e+04
Date:                Thu, 16 Nov 2017   Prob (F-

## Part (b)

In [7]:
# Part (b) -- Long Regression


# Define covariates
X_list2 = X_list1 + ['Father_NoSchool','Father_Complete1stPrimary',\
'Father_Complete2ndPrimary','Father_CompleteSecondary','Mother_NoSchool',\
'Mother_Complete1stPrimary','Mother_Complete2ndPrimary',\
'Mother_CompleteSecondary'];
X2 = data[X_list2];
X2 = sm.add_constant(X2);

# Compute OLS fit
result_long = OLS(Y,X2);

print '\n'
print 'Part (b): Long Regression'
print result_long[0]



Part (b): Long Regression
                               Coef        SE               95% CI
const                      3.021391  0.041864    [2.93934,3.10344]
YRSSCH                     0.136991  0.000937    [0.13515,0.13883]
AgeInDays                  0.091668  0.002364    [0.08704,0.09630]
AgeInDays2                -0.000927  0.000031  [-0.00099,-0.00087]
Father_NoSchool           -0.052852  0.009228  [-0.07094,-0.03477]
Father_Complete1stPrimary  0.080203  0.011191    [0.05827,0.10214]
Father_Complete2ndPrimary -0.045733  0.019222  [-0.08341,-0.00806]
Father_CompleteSecondary  -0.000254  0.021430   [-0.04226,0.04175]
Mother_NoSchool           -0.017216  0.009081   [-0.03501,0.00058]
Mother_Complete1stPrimary  0.117530  0.011506    [0.09498,0.14008]
Mother_Complete2ndPrimary  0.117912  0.017989    [0.08265,0.15317]
Mother_CompleteSecondary   0.212757  0.020444    [0.17269,0.25283]


In [9]:
# Part (c) Residual Regression

# Define the dependent variable and covariates for auxliary regression
yrssch = data[['YRSSCH']];

X_list3 = X_list2[:];
X_list3.remove('YRSSCH');
X3 = data[X_list3];
X3 = sm.add_constant(X3);

# OLS fit of auxiliary regression
coef_aux = la.lstsq(X3,yrssch)[0];

# residual of auxiliary regression
resid_aux = yrssch - np.dot(X3,coef_aux);

# OLS fit of residual regression
coef_resid = la.lstsq(resid_aux,Y)[0];

print '\n'
print 'The residual regression coefficient is %f' %coef_resid



The residual regression coefficient is 0.136991


The residual regresssion coefficient found here, is same as part (b) which is  0.136991 

## Part (d)

In [11]:
# Part (d) Bayesian Bootstrap

# Setting up parameters
N = len(X1); # Length of the weight array
NS = 1000; # Number of simulations

bb1 = []
bb2 = []

for j in range(NS):
    
    # Draw N-array of Gamma(1,1) random variables
    W = np.random.gamma(1,1,N);
    
    # Compute weighted OLS fit
    result1 = sm.WLS(Y,X1,weights=W).fit();
    result2 = sm.WLS(Y,X2,weights=W).fit();
    
    # Store result from each simulation
    bb1 = bb1 + [result1.params[1]];
    bb2 = bb2 + [result2.params[1]];

# Combine into a Pandas DataFrame
bb = pd.DataFrame({'Part (a)':bb1, 'Part (b)':bb2});

print '\n'
print 'Part (d): Bayesian Bootstrapping'
print bb.describe()




Part (d): Bayesian Bootstrapping
          Part (a)     Part (b)
count  1000.000000  1000.000000
mean      0.145565     0.136939
std       0.000846     0.000962
min       0.142789     0.133900
25%       0.144971     0.136275
50%       0.145574     0.136918
75%       0.146116     0.137600
max       0.148717     0.140046


We see that the posteriori distribution is close to asymptotic sampling distributions in (a) an (b). 